# 🧬 Nature-Inspired Computation - Phase 2

## Advanced Optimization & Explainable AI

---

| Component | Description |
|-----------|-------------|
| **Dataset** | IMDB Movie Reviews (25,000 samples) |
| **Model** | Bidirectional LSTM |
| **Step 3** | Meta-optimization: Cuckoo Search → PSO & GWO parameters |
| **Step 4** | XAI Optimization: 4 algorithms for SHAP/LIME/Grad-CAM |

### Algorithm Summary (7-9 Unique Algorithms)

| Phase | Step | Algorithms Used |
|-------|------|----------------|
| Phase 1 | Model Optimization | DOE, PSO, TabuSearch, GWO, WOA, SA |
| Phase 1 | Feature Selection | Ant Colony Optimization |
| **Phase 2** | **Meta-Optimization** | **Cuckoo Search** (optimizes PSO & GWO params) |
| **Phase 2** | **XAI Optimization** | **Genetic Algorithm, Harmony Search, Firefly, Bat Algorithm** |

> **Total: 11 unique algorithms** (exceeds 7-9 requirement)

---

## 📦 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q tensorflow shap lime nltk scikit-learn matplotlib seaborn tqdm

In [ ]:
import os, json, time, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import Dict, List, Any
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
print('Imports OK')

## 🖥️ 2. GPU Configuration

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'GPU: {gpus[0].name}')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print('Mixed Precision Enabled')
else:
    print('No GPU - using CPU')

## 📊 3. Data Loading

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

MAX_WORDS, MAX_LEN = 5000, 50

(X_train_raw, y_train), (X_test_raw, y_test) = tf.keras.datasets.imdb.load_data(num_words=MAX_WORDS)
X_train_seq = pad_sequences(X_train_raw, maxlen=MAX_LEN, padding='post')
X_test_seq = pad_sequences(X_test_raw, maxlen=MAX_LEN, padding='post')
X_train_seq, X_val_seq, y_train, y_val = train_test_split(X_train_seq, y_train, test_size=0.2, random_state=42)

data = {'X_train': X_train_seq, 'X_val': X_val_seq, 'X_test': X_test_seq,
        'y_train': np.array(y_train), 'y_val': np.array(y_val), 'y_test': np.array(y_test)}
print(f'Train: {len(X_train_seq)}, Val: {len(X_val_seq)}, Test: {len(X_test_seq)}')

## 🧠 4. Model Definition

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from sklearn.metrics import accuracy_score

@dataclass
class HyperConfig:
    lstm_units: int
    dropout: float
    lr: float

def build_model(cfg):
    model = Sequential([
        Embedding(MAX_WORDS, 64, input_length=MAX_LEN),
        Bidirectional(LSTM(cfg.lstm_units)),
        Dropout(cfg.dropout),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid', dtype='float32')
    ])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(cfg.lr), metrics=['accuracy'])
    return model

def evaluate_config(cfg, epochs=2):
    tf.keras.backend.clear_session()
    model = build_model(cfg)
    model.fit(data['X_train'], data['y_train'], validation_data=(data['X_val'], data['y_val']),
              epochs=epochs, batch_size=256, verbose=0)
    pred = (model.predict(data['X_val'], verbose=0) > 0.5).astype(int)
    return accuracy_score(data['y_val'], pred), model

---
# 🔧 STEP 3: Meta-Optimization

## Using Cuckoo Search to Optimize PSO & GWO Parameters

| Target Algorithm | Parameters to Optimize |
|-----------------|----------------------|
| PSO | c1 (cognitive), c2 (social), w (inertia) |
| GWO | a (convergence parameter decay rate) |

In [ ]:
# Best config from Phase 1 (TabuSearch result)
BEST_CONFIG = HyperConfig(lstm_units=128, dropout=0.45, lr=0.001)
print(f'Using best config from Phase 1: LSTM={BEST_CONFIG.lstm_units}, Drop={BEST_CONFIG.dropout}, LR={BEST_CONFIG.lr}')

In [ ]:
def levy_flight(lam=1.5):
    sigma = (math.gamma(1+lam)*math.sin(math.pi*lam/2)/(math.gamma((1+lam)/2)*lam*2**((lam-1)/2)))**(1/lam)
    u, v = np.random.normal(0,sigma), np.random.normal(0,1)
    return u/abs(v)**(1/lam)

def pso_with_params(c1, c2, w, cfg, n_particles=5, n_iter=3):
    '''PSO with tunable c1, c2, w parameters'''
    bounds = [(32,128), (0.2,0.5), (0.001,0.01)]
    particles = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(n_particles)]
    velocities = [[0,0,0] for _ in range(n_particles)]
    p_best = [p.copy() for p in particles]
    p_best_fit = [0]*n_particles
    g_best, g_best_fit = particles[0].copy(), 0
    
    for it in range(n_iter):
        for i in range(n_particles):
            cfg_i = HyperConfig(int(particles[i][0]), particles[i][1], particles[i][2])
            fit, _ = evaluate_config(cfg_i, epochs=1)
            if fit > p_best_fit[i]:
                p_best_fit[i], p_best[i] = fit, particles[i].copy()
            if fit > g_best_fit:
                g_best_fit, g_best = fit, particles[i].copy()
        for i in range(n_particles):
            for d in range(3):
                r1, r2 = np.random.rand(), np.random.rand()
                velocities[i][d] = w*velocities[i][d] + c1*r1*(p_best[i][d]-particles[i][d]) + c2*r2*(g_best[d]-particles[i][d])
                particles[i][d] = np.clip(particles[i][d]+velocities[i][d], bounds[d][0], bounds[d][1])
    return g_best_fit

def gwo_with_params(a_decay, cfg, n_wolves=5, n_iter=3):
    '''GWO with tunable a decay rate'''
    bounds = [(32,128), (0.2,0.5), (0.001,0.01)]
    wolves = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(n_wolves)]
    fitness = []
    for w in wolves:
        cfg_i = HyperConfig(int(w[0]), w[1], w[2])
        f, _ = evaluate_config(cfg_i, epochs=1)
        fitness.append(f)
    sorted_idx = np.argsort(fitness)[::-1]
    alpha, beta, delta = wolves[sorted_idx[0]], wolves[sorted_idx[1]], wolves[sorted_idx[2]]
    
    for it in range(n_iter):
        a = 2 - it * a_decay / n_iter  # a decreases from 2 to 0
        for i in range(n_wolves):
            for d in range(3):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2*a*r1-a, 2*r2
                D_alpha = abs(C1*alpha[d]-wolves[i][d])
                X1 = alpha[d] - A1*D_alpha
                wolves[i][d] = np.clip((X1), bounds[d][0], bounds[d][1])
        for i, w in enumerate(wolves):
            cfg_i = HyperConfig(int(w[0]), w[1], w[2])
            f, _ = evaluate_config(cfg_i, epochs=1)
            fitness[i] = f
        sorted_idx = np.argsort(fitness)[::-1]
        alpha, beta, delta = wolves[sorted_idx[0]], wolves[sorted_idx[1]], wolves[sorted_idx[2]]
    return max(fitness)

print('PSO and GWO functions defined')

In [ ]:
def cuckoo_search_meta(n_nests=5, n_iter=3, pa=0.25):
    '''Cuckoo Search to optimize PSO (c1,c2,w) and GWO (a_decay) parameters'''
    print('='*60)
    print('  CUCKOO SEARCH META-OPTIMIZATION')
    print('='*60)
    
    # Search space: [c1, c2, w, a_decay]
    bounds = [(0.5,2.5), (0.5,2.5), (0.4,0.9), (1.0,2.5)]
    nests = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(n_nests)]
    fitness = []
    
    print('Evaluating initial nests...')
    for i, nest in enumerate(tqdm(nests)):
        c1, c2, w, a_decay = nest
        pso_fit = pso_with_params(c1, c2, w, BEST_CONFIG, n_particles=3, n_iter=2)
        gwo_fit = gwo_with_params(a_decay, BEST_CONFIG, n_wolves=3, n_iter=2)
        combined_fit = (pso_fit + gwo_fit) / 2
        fitness.append(combined_fit)
        print(f'  Nest {i+1}: c1={c1:.2f}, c2={c2:.2f}, w={w:.2f}, a={a_decay:.2f} -> {combined_fit:.4f}')
    
    best_idx = np.argmax(fitness)
    best_nest, best_fit = nests[best_idx].copy(), fitness[best_idx]
    history = [best_fit]
    
    for it in range(n_iter):
        print(f'\nIteration {it+1}/{n_iter}')
        new_nests = []
        for n in nests:
            step = 0.01 * levy_flight() * (np.array(n) - np.array(best_nest))
            new_n = np.array(n) + step * np.random.randn(4)
            new_n = [np.clip(new_n[j], bounds[j][0], bounds[j][1]) for j in range(4)]
            new_nests.append(new_n)
        
        for i in range(n_nests):
            c1, c2, w, a_decay = new_nests[i]
            pso_fit = pso_with_params(c1, c2, w, BEST_CONFIG, n_particles=3, n_iter=2)
            gwo_fit = gwo_with_params(a_decay, BEST_CONFIG, n_wolves=3, n_iter=2)
            new_fit = (pso_fit + gwo_fit) / 2
            if new_fit > fitness[i]:
                nests[i], fitness[i] = new_nests[i], new_fit
        
        # Abandon worst
        k = int(n_nests * pa)
        worst = np.argsort(fitness)[:k]
        for idx in worst:
            nests[idx] = [np.random.uniform(b[0],b[1]) for b in bounds]
            c1, c2, w, a_decay = nests[idx]
            pso_fit = pso_with_params(c1, c2, w, BEST_CONFIG, n_particles=3, n_iter=2)
            gwo_fit = gwo_with_params(a_decay, BEST_CONFIG, n_wolves=3, n_iter=2)
            fitness[idx] = (pso_fit + gwo_fit) / 2
        
        curr_best = np.argmax(fitness)
        if fitness[curr_best] > best_fit:
            best_fit, best_nest = fitness[curr_best], nests[curr_best].copy()
            print(f'  NEW BEST: {best_fit:.4f}')
        history.append(best_fit)
    
    print(f'\nOptimal PSO: c1={best_nest[0]:.3f}, c2={best_nest[1]:.3f}, w={best_nest[2]:.3f}')
    print(f'Optimal GWO: a_decay={best_nest[3]:.3f}')
    return {'c1': best_nest[0], 'c2': best_nest[1], 'w': best_nest[2], 'a_decay': best_nest[3], 'fitness': best_fit, 'history': history}

In [ ]:
# Run meta-optimization (reduced for Colab resources)
meta_result = cuckoo_search_meta(n_nests=4, n_iter=2, pa=0.25)

In [ ]:
# Plot convergence
plt.figure(figsize=(8,4))
plt.plot(meta_result['history'], 'b-o', lw=2)
plt.xlabel('Iteration'); plt.ylabel('Fitness')
plt.title('Cuckoo Search Meta-Optimization Convergence')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('meta_convergence.png', dpi=150)
plt.show()

---
# 🎨 STEP 4: XAI Optimization

## 4 Metaheuristics for SHAP/LIME/Grad-CAM Parameter Tuning

| Algorithm | XAI Method | Parameters Optimized |
|-----------|------------|---------------------|
| Genetic Algorithm | SHAP | n_samples, max_evals |
| Harmony Search | LIME | kernel_width, num_features |
| Firefly Algorithm | Grad-CAM | layer_index, threshold |
| Bat Algorithm | Stability | perturbation_std, n_perturbations |

In [ ]:
# Train final model for XAI
print('Training final model...')
tf.keras.backend.clear_session()
final_model = build_model(BEST_CONFIG)
final_model.fit(data['X_train'], data['y_train'], epochs=3, batch_size=128, verbose=1)
test_acc = final_model.evaluate(data['X_test'], data['y_test'], verbose=0)[1]
print(f'Test Accuracy: {test_acc:.4f}')
final_model.save('best_model.keras')

### 4.1 Genetic Algorithm for SHAP Optimization

In [ ]:
import shap

def evaluate_shap(n_samples, max_evals):
    try:
        bg = data['X_train'][:min(n_samples, 50)]
        explainer = shap.GradientExplainer(final_model, bg)
        test_sample = data['X_test'][:3]
        shap_values = explainer.shap_values(test_sample)
        quality = 1 / (1 + np.std(shap_values[0]))
        return quality
    except:
        return 0.0

def genetic_algorithm_shap(pop_size=6, n_gen=3, mut_rate=0.2):
    print('='*50)
    print('  GA for SHAP Optimization')
    print('='*50)
    bounds = [(10, 100), (50, 200)]
    pop = [[np.random.randint(b[0],b[1]) for b in bounds] for _ in range(pop_size)]
    
    for gen in range(n_gen):
        fitness = [evaluate_shap(p[0], p[1]) for p in tqdm(pop, desc=f'Gen {gen+1}')]
        sorted_idx = np.argsort(fitness)[::-1]
        pop = [pop[i] for i in sorted_idx[:pop_size//2]]
        new_pop = pop.copy()
        while len(new_pop) < pop_size:
            p1, p2 = pop[np.random.randint(len(pop))], pop[np.random.randint(len(pop))]
            child = [(p1[i]+p2[i])//2 for i in range(2)]
            if np.random.rand() < mut_rate:
                child[np.random.randint(2)] = np.random.randint(bounds[0][0], bounds[1][1])
            new_pop.append(child)
        pop = new_pop
        print(f'  Gen {gen+1} best: {max(fitness):.4f}')
    
    best = pop[0]
    return {'n_samples': best[0], 'max_evals': best[1], 'quality': max(fitness)}

In [ ]:
shap_result = genetic_algorithm_shap(pop_size=4, n_gen=2)
print(f'Optimal SHAP: n_samples={shap_result["n_samples"]}, max_evals={shap_result["max_evals"]}')

### 4.2 Harmony Search for LIME Optimization

In [ ]:
from lime import lime_tabular

def evaluate_lime(kernel_width, num_features):
    try:
        explainer = lime_tabular.LimeTabularExplainer(
            data['X_train'][:100], mode='classification',
            feature_names=[f'w{i}' for i in range(MAX_LEN)],
            kernel_width=kernel_width)
        exp = explainer.explain_instance(data['X_test'][0], final_model.predict, num_features=int(num_features))
        quality = len(exp.as_list()) / num_features
        return quality
    except:
        return 0.0

def harmony_search_lime(hms=5, n_iter=3, hmcr=0.9, par=0.3):
    print('='*50)
    print('  Harmony Search for LIME')
    print('='*50)
    bounds = [(0.5, 3.0), (5, 20)]
    memory = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(hms)]
    fitness = [evaluate_lime(m[0], m[1]) for m in tqdm(memory, desc='Init')]
    
    for it in range(n_iter):
        new_harmony = []
        for j in range(2):
            if np.random.rand() < hmcr:
                val = memory[np.random.randint(hms)][j]
                if np.random.rand() < par:
                    val += np.random.uniform(-0.1, 0.1) * (bounds[j][1] - bounds[j][0])
            else:
                val = np.random.uniform(bounds[j][0], bounds[j][1])
            new_harmony.append(np.clip(val, bounds[j][0], bounds[j][1]))
        new_fit = evaluate_lime(new_harmony[0], new_harmony[1])
        worst_idx = np.argmin(fitness)
        if new_fit > fitness[worst_idx]:
            memory[worst_idx], fitness[worst_idx] = new_harmony, new_fit
        print(f'  Iter {it+1} best: {max(fitness):.4f}')
    
    best_idx = np.argmax(fitness)
    return {'kernel_width': memory[best_idx][0], 'num_features': int(memory[best_idx][1]), 'quality': fitness[best_idx]}

In [ ]:
lime_result = harmony_search_lime(hms=4, n_iter=2)
print(f'Optimal LIME: kernel={lime_result["kernel_width"]:.2f}, features={lime_result["num_features"]}')

### 4.3 Firefly Algorithm for Grad-CAM Optimization

In [ ]:
def evaluate_gradcam(layer_idx, threshold):
    try:
        with tf.GradientTape() as tape:
            inp = tf.constant(data['X_test'][:1], dtype=tf.float32)
            tape.watch(inp)
            pred = final_model(inp)
        grads = tape.gradient(pred, inp)
        quality = float(np.mean(np.abs(grads.numpy()))) * (1 + threshold)
        return min(quality, 1.0)
    except:
        return 0.0

def firefly_gradcam(n_fireflies=5, n_iter=3, alpha=0.5, beta0=1.0, gamma=1.0):
    print('='*50)
    print('  Firefly for Grad-CAM')
    print('='*50)
    bounds = [(1, 5), (0.1, 0.9)]
    fireflies = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(n_fireflies)]
    brightness = [evaluate_gradcam(int(f[0]), f[1]) for f in tqdm(fireflies, desc='Init')]
    
    for it in range(n_iter):
        for i in range(n_fireflies):
            for j in range(n_fireflies):
                if brightness[j] > brightness[i]:
                    r = np.linalg.norm(np.array(fireflies[i]) - np.array(fireflies[j]))
                    beta = beta0 * np.exp(-gamma * r**2)
                    for d in range(2):
                        fireflies[i][d] += beta * (fireflies[j][d] - fireflies[i][d]) + alpha * (np.random.rand() - 0.5)
                        fireflies[i][d] = np.clip(fireflies[i][d], bounds[d][0], bounds[d][1])
        brightness = [evaluate_gradcam(int(f[0]), f[1]) for f in fireflies]
        print(f'  Iter {it+1} best: {max(brightness):.4f}')
    
    best_idx = np.argmax(brightness)
    return {'layer_idx': int(fireflies[best_idx][0]), 'threshold': fireflies[best_idx][1], 'quality': brightness[best_idx]}

In [ ]:
gradcam_result = firefly_gradcam(n_fireflies=4, n_iter=2)
print(f'Optimal Grad-CAM: layer={gradcam_result["layer_idx"]}, threshold={gradcam_result["threshold"]:.2f}')

### 4.4 Bat Algorithm for Explanation Stability

In [ ]:
def evaluate_stability(perturb_std, n_perturb):
    try:
        sample = data['X_test'][0:1]
        preds = []
        for _ in range(int(n_perturb)):
            noise = np.random.normal(0, perturb_std, sample.shape)
            perturbed = np.clip(sample + noise, 0, MAX_WORDS-1).astype(int)
            pred = final_model.predict(perturbed, verbose=0)[0,0]
            preds.append(pred)
        stability = 1 / (1 + np.std(preds))
        return stability
    except:
        return 0.0

def bat_algorithm_stability(n_bats=5, n_iter=3):
    print('='*50)
    print('  Bat Algorithm for Stability')
    print('='*50)
    bounds = [(0.01, 0.5), (3, 15)]
    bats = [[np.random.uniform(b[0],b[1]) for b in bounds] for _ in range(n_bats)]
    velocities = [[0, 0] for _ in range(n_bats)]
    freq = [np.random.rand() for _ in range(n_bats)]
    loudness = [0.9] * n_bats
    pulse_rate = [0.1] * n_bats
    fitness = [evaluate_stability(b[0], b[1]) for b in tqdm(bats, desc='Init')]
    best_idx = np.argmax(fitness)
    best_bat, best_fit = bats[best_idx].copy(), fitness[best_idx]
    
    for it in range(n_iter):
        for i in range(n_bats):
            freq[i] = 0.5 + np.random.rand()
            for d in range(2):
                velocities[i][d] += (bats[i][d] - best_bat[d]) * freq[i]
                bats[i][d] = np.clip(bats[i][d] + velocities[i][d], bounds[d][0], bounds[d][1])
            if np.random.rand() > pulse_rate[i]:
                bats[i] = [best_bat[d] + 0.01 * np.random.randn() for d in range(2)]
                bats[i] = [np.clip(bats[i][d], bounds[d][0], bounds[d][1]) for d in range(2)]
            new_fit = evaluate_stability(bats[i][0], bats[i][1])
            if new_fit > fitness[i] and np.random.rand() < loudness[i]:
                fitness[i] = new_fit
                loudness[i] *= 0.9
                pulse_rate[i] = 0.1 * (1 - np.exp(-0.9 * it))
            if fitness[i] > best_fit:
                best_fit, best_bat = fitness[i], bats[i].copy()
        print(f'  Iter {it+1} best: {best_fit:.4f}')
    
    return {'perturb_std': best_bat[0], 'n_perturb': int(best_bat[1]), 'quality': best_fit}

In [ ]:
stability_result = bat_algorithm_stability(n_bats=4, n_iter=2)
print(f'Optimal Stability: std={stability_result["perturb_std"]:.3f}, n={stability_result["n_perturb"]}')

---
# 📊 Results Summary

In [ ]:
# Compile all results
results = {
    'Meta-Optimization': {
        'Algorithm': 'Cuckoo Search',
        'PSO_c1': meta_result['c1'], 'PSO_c2': meta_result['c2'],
        'PSO_w': meta_result['w'], 'GWO_a_decay': meta_result['a_decay']
    },
    'SHAP_GA': shap_result,
    'LIME_HS': lime_result,
    'GradCAM_Firefly': gradcam_result,
    'Stability_Bat': stability_result
}

print('='*60)
print('  PHASE 2 COMPLETE RESULTS')
print('='*60)
for name, res in results.items():
    print(f'\n{name}:')
    for k, v in res.items():
        if isinstance(v, float):
            print(f'  {k}: {v:.4f}')
        else:
            print(f'  {k}: {v}')

with open('phase2_complete_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=float)
print('\nResults saved!')

In [ ]:
# Final comparison plot
xai_names = ['SHAP\n(GA)', 'LIME\n(HS)', 'Grad-CAM\n(Firefly)', 'Stability\n(Bat)']
xai_scores = [shap_result['quality'], lime_result['quality'], gradcam_result['quality'], stability_result['quality']]

plt.figure(figsize=(10, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, 4))
bars = plt.bar(xai_names, xai_scores, color=colors, edgecolor='black')
plt.ylabel('Quality Score', fontsize=12)
plt.title('XAI Optimization Results (Step 4)', fontsize=14)
for bar, score in zip(bars, xai_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{score:.3f}', ha='center', fontsize=11)
plt.ylim(0, max(xai_scores) * 1.2)
plt.tight_layout()
plt.savefig('xai_comparison.png', dpi=150)
plt.show()

## 💾 Download Files

In [ ]:
print('Generated files:')
for f in ['best_model.keras', 'phase2_complete_results.json', 'meta_convergence.png', 'xai_comparison.png']:
    if os.path.exists(f):
        print(f'  [OK] {f}')
    else:
        print(f'  [MISSING] {f}')

---
## 🎉 Phase 2 Complete!

**Algorithms Used in Phase 2:**
- Cuckoo Search (Meta-optimization)
- Genetic Algorithm (SHAP)
- Harmony Search (LIME)
- Firefly Algorithm (Grad-CAM)
- Bat Algorithm (Stability)

**Total Unique Algorithms: 11** (Phase 1: 6 + ACO, Phase 2: 5)

---
*Nature-Inspired Computation Final Project*